# ELEC 598 : Deep Learning Applications 
# Project 1 - Predicting Housing Prices 
### By: Matthew Stein and Ethan Tsao 
### February 10, 2020 

In [ ]:
from keras import models
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()f

## Data Preprocessing 

Data is imported via pandas, and processed via numpy. Working with Keras, all data is processed as arrays in numpy. To simplify model learning, feature-wise normalization is performed. Note that feature-wise normalization is calc'ed using the mean and std of the training set. 

The test set was chosen to be 15% of the overall data set. 

In [26]:
data = pd.read_excel('data/resd_data.xlsx')
length = data.shape[0]

In [27]:
train_data = data.loc[:8500,:'field_PostalCode'].values.tolist()
train_data = np.asarray(train_data) 
train_targets = data.loc[:8500,'ListPrice'].values.tolist()
train_targets = np.asarray(train_targets) 

test_data = data.loc[8501:,:'field_PostalCode'].values.tolist()
test_data = np.asarray(test_data) 
test_targets = data.loc[8501:,'ListPrice'].values.tolist()
test_targets = np.asarray(test_targets)

In [28]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

## Model Selection 

As is standard for neural networks - and regression models, in particular - a Sequential model was chosen. The model has 3 hidden layers, with 64 

In [36]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

## Training and Validation 

In [37]:
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_mae_histories = []


for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    print(train_data[:i * num_val_samples].shape)
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=1)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    #mae_history = history.history['val_mean_absolute_error']
    #all_mae_histories.append(mae_history)

processing fold # 0
(0, 7)
Train on 6376 samples, validate on 2125 samples
Epoch 1/100
4384/6376 [===================>..........] - ETA: 3s - loss: nan - mae: nan

KeyboardInterrupt: 

## Results Discussion 